In [46]:
import pandas as pd # read the csv
import re # regex to detect username, url, html entity 
import nltk # to use word tokenize (split the sentence into words)
from nltk.corpus import stopwords # to remove the stopwords
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras import backend as K

In [47]:
# Load the dataset
df = pd.read_csv('./labeled_data.csv')
df.head()

# extract the text and labels
tweet = list(df['tweet'])
labels = list(df['class'])

### Steps:
1. preprocessing
2. splitting
3. tokenize & padding
4. Create model & train
5. evaluate

---

Preprocessing (cleaning the datasets):
- remove html entity
- change user tags (@xxx -> user)
- remove urls
- remove unnecessary  symbol ('', !, ", ') -> cause a lot of noise in the dataset
- remove stopwords

In [48]:
## notes: all of the function taking 1 text at a time
stop_words = set(stopwords.words('english'))
# add rt to remove retweet in dataset (noise)
stop_words.add("rt")

# remove html entity:
def remove_entity(raw_text):
    entity_regex = r"&[^\s;]+;"
    text = re.sub(entity_regex, "", raw_text)
    return text

# change the user tags
def change_user(raw_text):
    regex = r"@([^ ]+)"
    text = re.sub(regex, "user", raw_text)

    return text

# remove urls
def remove_url(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_regex, '', raw_text)

    return text

# remove unnecessary symbols
def remove_noise_symbols(raw_text):
    text = raw_text.replace('"', '')
    text = text.replace("'", '')
    text = text.replace("!", '')
    text = text.replace("`", '')
    text = text.replace("..", '')

    return text

# remove stopwords
def remove_stopwords(raw_text):
    tokenize = nltk.word_tokenize(raw_text)
    text = [word for word in tokenize if not word.lower() in stop_words]
    text = " ".join(text)

    return text

## this function in to clean all the dataset by utilizing all the function above
def preprocess(datas):
    clean = []
    # change the @xxx into "user"
    clean = [change_user(text) for text in datas]
    # remove emojis (specifically unicode emojis)
    clean = [remove_entity(text) for text in clean]
    # remove urls
    clean = [remove_url(text) for text in clean]
    # remove trailing stuff
    clean = [remove_noise_symbols(text) for text in clean]
    # remove stopwords
    clean = [remove_stopwords(text) for text in clean]

    return clean

In [49]:
# call the cleaning function
clean_tweet = preprocess(tweet)

Splitting the dataset into test and validation

In [50]:
X_train, X_test, y_train, y_test = train_test_split(clean_tweet, labels, test_size=0.2, random_state=42)

Tokenizing and Padding

In [51]:
## Tokenizing -> basically we use tokenisation for many things, its commonly used for feature extraction in preprocessing. btw idk how it works as feature extraction tho :(
# declare the tokenizer
tokenizer = Tokenizer()
# build the vocabulary based on train dataset
tokenizer.fit_on_texts(X_train)
# tokenize the train and test dataset
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# vocabulary size (num of unique words) -> will be used in embedding layer
vocab_size = len(tokenizer.word_index) + 1

In [52]:
## Padding -> to uniform the datas
max_length = max(len(seq) for seq in X_train)

# to test an outlier case (if one of the test dataset has longer length)
for x in X_test:
    if len(x) > max_length:
        print(f"an outlier detected: {x}")

X_train = pad_sequences(X_train, maxlen = max_length)
X_test = pad_sequences(X_test, maxlen = max_length)

In [58]:
# create hot_labels (idk whty tapi ini penting)
y_test = to_categorical(y_test, num_classes=3)
y_train = to_categorical(y_train, num_classes=3)

Model building

In [59]:
# evaluation metrics
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [60]:
# LSTM model architechture (CNN + LSTM)
model = Sequential([
    # embedding layer is like idk
    Embedding(vocab_size, 100, input_length=max_length),
    # lstm for xxx
    LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    # dropout to prevent overfitting
    Dropout(0.5),
    # dense to connect the previous output with current layer
    Dense(128, activation="relu"),
    # dropout to prevent overfitting
    Dropout(0.5),
    # this is output layer, with 3 class (0, 1, 2)
    Dense(3, activation="softmax"),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1,precision, recall])


In [61]:
# Train the model
model_history = model.fit(
    X_train,
    y_train,
    batch_size = 32,
    epochs=10,
    validation_data=(X_test, y_test)
)

Epoch 1/10
620/620 [==============================] - 16s 23ms/step - loss: 0.4314 - accuracy: 0.8523 - f1: 0.8372 - precision: 0.8628 - recall: 0.8163 - val_loss: 0.3194 - val_accuracy: 0.8937 - val_f1: 0.8925 - val_precision: 0.9087 - val_recall: 0.8774
Epoch 2/10
620/620 [==============================] - 13s 21ms/step - loss: 0.2336 - accuracy: 0.9218 - f1: 0.9216 - precision: 0.9316 - recall: 0.9122 - val_loss: 0.3078 - val_accuracy: 0.8957 - val_f1: 0.8941 - val_precision: 0.9035 - val_recall: 0.8852
Epoch 3/10
620/620 [==============================] - 13s 21ms/step - loss: 0.1537 - accuracy: 0.9451 - f1: 0.9451 - precision: 0.9489 - recall: 0.9415 - val_loss: 0.3941 - val_accuracy: 0.8806 - val_f1: 0.8802 - val_precision: 0.8855 - val_recall: 0.8751
Epoch 4/10
620/620 [==============================] - 13s 21ms/step - loss: 0.1067 - accuracy: 0.9631 - f1: 0.9630 - precision: 0.9643 - recall: 0.9618 - val_loss: 0.5024 - val_accuracy: 0.8685 - val_f1: 0.8688 - val_precision: 0.87